In [1]:
import numpy as np
import pandas as pd
import math
from math import sin, cos, radians
import random
import re

import os
import matplotlib.pyplot as plt
import datetime

import scipy.stats as st
import scipy.signal as sgl
pd.set_option('display.max_columns', 500)

#import fastdtw
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

import scipy.interpolate as spi

#hide warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
# """
# The Ramer-Douglas-Peucker algorithm roughly ported from the pseudo-code provided
# by http://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm
# """

# from math import sqrt

# def distance(a, b):
#     return  sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

# def point_line_distance(point, start, end):
#     if (start == end):
#         return distance(point, start)
#     else:
#         n = abs(
#             (end[0] - start[0]) * (start[1] - point[1]) - (start[0] - point[0]) * (end[1] - start[1])
#         )
#         d = sqrt(
#             (end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2
#         )
#         return n / d

# def rdp(points, epsilon):
#     """
#     Reduces a series of points to a simplified version that loses detail, but
#     maintains the general shape of the series.
#     """
#     dmax = 0.0
#     index = 0
#     for i in range(1, len(points) - 1):
#         d = point_line_distance(points[i], points[0], points[-1])
#         if d > dmax:
#             index = i
#             dmax = d
#     if dmax >= epsilon:
#         results = rdp(points[:index+1], epsilon)[:-1] + rdp(points[index:], epsilon)
#     else:
#         results = [points[0], points[-1]]
#     return results


# def rdp_precoss(df):
#     l = []
#     for i in range(0, df.shape[0]):
#         l.append((df.loc[i, 'X'], df.loc[i, 'Y'],  df.loc[i, 'TStamp'], df.loc[i, 'Pres'], df.loc[i, 'EndPts'],  df.loc[i, 'TStamp2'] ))

#     final = rdp(l, 0.0000001)
# #     print (f)
#     df = pd.DataFrame(final, columns=['X', 'Y', 'TStamp', 'Pres', 'EndPts', 'TStamp2'])
#     return df

In [3]:
# remove_duplicated_point
def remove_duplicated_point(df):
    df_new = df.drop(index=df.index)    
    old_x = df['X'][0]
    old_y = df['Y'][0]
    for index, row in df.iterrows():
        if row['X'] != old_x or row['Y']!=old_y or index == 0:
            df_new.loc[len(df_new)] = {'X': row['X'], 'Y': row['Y'], 'TStamp': row['TStamp'], 'Pres': row['Pres'], 'EndPts': row['EndPts'],
                                       'TStamp2': row['TStamp2']}
            old_x = row['X']
            old_y = row['Y']
        # update the EndPts if the point is   
        elif row['X'] == old_x and row['Y'] == old_y and row['EndPts'] == 1:
            df_new.iloc[len(df_new)-1]['EndPts']  = 1
    return df_new

In [4]:
# # Cubic-Spline to add points in stroke and make the curves smooth
# def interpolate_points(df):
#     plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
#     plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

#     # 按手指离开屏幕的点，分笔触差值，因为在笔触间差值没有意义
#     # 寻找断点
#     EndPts = [0]
#     EndPts.extend(list(df.loc[df["EndPts"] == 1].index))
#     EndPts_len = len(EndPts)
# #     print(EndPts)
#     if EndPts_len < 2:
#         EndPts.append(len(df)-1) #如果没有就设最后一个点为离开屏幕的点
#         EndPts_len += 1
    
#     i = 0
#     new_x_all = []
#     iy3_x_all = []
#     iy3_y_all = []
#     print(EndPts)
#     while i < EndPts_len - 1:
#         #data preparation
#         start = EndPts[i]
#         end = EndPts[i+1]
#         if start == 0:
#             X= np.array(df['TStamp2'][0:end].tolist())
#             Y= np.array(df['X'][0:end].tolist())
#             Y2= np.array(df['Y'][0:end].tolist())
#             if df['TStamp2'][start+1:end].max() - df['TStamp2'][start+1:end].min() <=15 or end - start <= 6:
#                 i+=1
#                 continue            
#             new_x=np.arange(0,df['TStamp2'][0:end].max(),15) #define interpolate points
            
#         else:            
#             X= np.array(df['TStamp2'][start+1:end].tolist())
#             Y= np.array(df['X'][start+1:end].tolist())
#             Y2= np.array(df['Y'][start+1:end].tolist())
#             if df['TStamp2'][start+1:end].max() - df['TStamp2'][start+1:end].min() <=15 or end - start <= 6:
#                 i += 1
#                 continue
#             new_x=np.arange(df['TStamp2'][start+1:end].min(),df['TStamp2'][start+1:end].max(),15) #define interpolate points            


#         # 三次样条拟合
          # Cubic spline fitting  
#         ipo3=spi.splrep(X,Y,k=1, s=10000) #样本点导入，生成参数
#         iy3=spi.splev(new_x,ipo3) #根据观测点和样条参数，生成插值
#         ipo3_y=spi.splrep(X,Y2,k=1, s=10000) #样本点导入，生成参数
#         iy3_y=spi.splev(new_x,ipo3_y) #根据观测点和样条参数，生成插值        
#         new_x_all.extend(new_x)
#         iy3_x_all.extend(iy3)
#         iy3_y_all.extend(iy3_y)
#         i += 1

#     X_all= np.array(df['TStamp2'].tolist())
#     Y_all = np.array(df['X'].tolist())
#     Y2_all = np.array(df['Y'].tolist())
       
#     df_new = df.copy()
#     for i, val in enumerate(new_x_all):
#         if val in df['TStamp2'].tolist():
#             continue
#         if val in df['TStamp2'].tolist():
#             continue
#         if  abs(iy3_x_all[i]) > 100000 or abs(iy3_y_all[i]) > 100000:
#             continue
#         df_new.loc[len(df_new)] = {'X': iy3_x_all[i], 'Y': iy3_y_all[i], 'TStamp': 0, 'Pres': 0, 'EndPts': 0,
#                                        'TStamp2': val}
#     df_new = df_new.sort_values(by=['TStamp2'])
#     return df_new

In [5]:
# 求两直线夹角
# Find the angle between two straight lines
# def get_angle_betw_lines(x1, y1, x2, y2, x3, y3, x4, y4):
#     k1 = (y2-y1)/(float(x2-x1))
#     k2 = (y4-y3)/(float(x4-x3))
#     Cobb = float(math.fabs(np.arctan((k1-k2)/(float(1 + k1*k2)))*180/np.pi)+0.5)
#     return Cobb

# def get_grivity_angle(P):
#     points_left = []
#     points_right = []
# #     print(list(P.exterior.coords))
#     for point in P.exterior.coords:
#         if point[0] <= P.centroid.x:
#             points_left.append([point[0], point[1]])
#         else:
#             points_right.append([point[0], point[1]])

#     if len(points_left) < 4:
#         print(list(P.exterior.coords))
#         print(P.centroid)
        
#     P_left = Polygon(points_left)
#     P_right = Polygon(points_right)        
# #     plt.plot(P_left.centroid.y, P_left.centroid.x, 'bx')
# #     plt.plot(P_right.centroid.y, P_right.centroid.x, 'bx')

#     theta = get_angle_betw_lines( P_left.centroid.x,  P_left.centroid.y,  P_right.centroid.x,  P_right.centroid.y,      
#                                  0.0,0.0, 10.0,0.0)
#     return theta


# #rotate for Polygon
# def rotate_polygon(polygon, angle, center_point=(0, 0)):
#     """Rotates the given polygon which consists of corners represented as (x,y)
#     around center_point (origin by default)
#     Rotation is counter-clockwise
#     Angle is in degrees
#     """
#     rotated_polygon = []
#     for corner in polygon.exterior.coords:
#         rotated_corner = rotate_point(corner, angle, center_point)
#         rotated_polygon.append(rotated_corner)
#     rotated_polygon = Polygon(rotated_polygon)    
#     return rotated_polygon

# def rotate_point(point, angle, center_point=(0, 0)):
#     """Rotates a point around center_point(origin by default)
#     Angle is in degrees.
#     Rotation is counter-clockwise
#     """
#     angle_rad = radians(angle % 360)
#     # Shift the point so that center_point becomes the origin
#     new_point = (point[0] - center_point[0], point[1] - center_point[1])
#     new_point = (new_point[0] * cos(angle_rad) - new_point[1] * sin(angle_rad),
#                  new_point[0] * sin(angle_rad) + new_point[1] * cos(angle_rad))
#     # Reverse the shifting we have done
#     new_point = (new_point[0] + center_point[0], new_point[1] + center_point[1])
#     return new_point



# # gravity_x, gravity_y = gravity_normalize(file)
# # print(gravity_x, gravity_y)

# from shapely.geometry import Polygon

# def rotate_graphic(file):
#     points_left = []
#     points_right = []
#     points = []

#     for index, row in file.iterrows():
#         points.append([row['X'], row['Y']])

#     P = Polygon(points)
# #     print(P.centroid)

#     theta = get_grivity_angle(P)
#     theta_old = theta
# #     print(theta)
    
#     limit_rotate_times = 100
#     times = 0
#     min_theta = theta
#     min_theta_times = 0
#     if theta >6 and get_grivity_angle(rotate_polygon(P, theta_old/24, (P.centroid.x, P.centroid.y))) < theta: 
#         while theta > 6.0 and times <= limit_rotate_times :
#     #     if theta <= 90: # clock-wise 顺时针
#             P = rotate_polygon(P, theta_old/24, (P.centroid.x, P.centroid.y))        
#             theta = get_grivity_angle(P)
#             if theta < min_theta:
#                 min_theta = theta
#                 min_theta_times = times
#             times += 1    
# #             print(theta)
#         #rotate back to min theta
#         if theta >6:
#             P = rotate_polygon(P, -theta_old/24*(limit_rotate_times-min_theta_times), (P.centroid.x, P.centroid.y))
#             theta = get_grivity_angle(P)
#             print(min_theta)
#             print(theta)
            
#     elif  theta >6 and get_grivity_angle(rotate_polygon(P, -theta_old/24, (P.centroid.x, P.centroid.y))) < theta_old:
#         while theta > 6.0 and times <= limit_rotate_times:
#             P = rotate_polygon(P, -theta_old/24, (P.centroid.x, P.centroid.y))        
#             theta = get_grivity_angle(P)
#             if theta < min_theta:
#                 min_theta = theta
#                 min_theta_times = times
#             times += 1    
# #             print(theta)
#         #rotate back to min theta
#         if theta >6:
#             P = rotate_polygon(P, theta_old/24*(limit_rotate_times-min_theta_times), (P.centroid.x, P.centroid.y))
#             theta = get_grivity_angle(P)
#             print(min_theta)
#             print(theta)
            
            
#     P_rotated = P
#     P_rotated_points_x = []
#     P_rotated_points_y = []
#     for point in   P_rotated.exterior.coords:
#         P_rotated_points_x.append(point[0])   
#         P_rotated_points_y.append(point[1])   

# #     print(P_rotated.centroid)




#     fig = plt.figure(figsize=[6,4])
#     plt.scatter(x = file["X"], y = file["Y"], marker='o',c='', edgecolors='g')

# #     plt.plot(gravity_y, gravity_x, 'rx')
# #     plt.plot(P_left.centroid.y, P_left.centroid.x, 'rx')
# #     plt.plot(P_right.centroid.y, P_right.centroid.x, 'rx')

#     plt.scatter(x = P_rotated_points_x, y = P_rotated_points_y, marker='o',c='', edgecolors='b')

#     points_left = []
#     points_right = []
#     for point in P_rotated.exterior.coords:
#         if point[0] <= P_rotated.centroid.x:
#             points_left.append([point[0], point[1]])
#         else:
#             points_right.append([point[0], point[1]])

#     P_left = Polygon(points_left)
#     P_right = Polygon(points_right)        
#     plt.plot(P_left.centroid.x, P_left.centroid.y, 'rx')
#     plt.plot(P_right.centroid.x, P_right.centroid.y, 'rx')

#     theta = get_angle_betw_lines( P_left.centroid.x,  P_left.centroid.y,  P_right.centroid.x,  P_right.centroid.y,      
#                                  0,0, 10,0)
# #     print(theta)
#     fig.show()

#     file['X'] = P_rotated_points_x[0:-1]
#     file['Y'] = P_rotated_points_y[0:-1]
#     return file

In [6]:
# Size normalization 大小规整
# position normalization 位置规整
def normalizeFile(normalInputFile):
    widthX = 500 #width
    heightY = 200 #height
    minX = normalInputFile['X'].min()
    minY = normalInputFile['Y'].min()
    maxX = normalInputFile['X'].max()
    maxY = normalInputFile['Y'].max()
    normalInputFile['normalX'] = widthX * ((normalInputFile['X'] - minX)/(maxX - minX))
    normalInputFile['normalY'] = heightY * ((normalInputFile['Y'] - minY)/(maxY - minY))
    
    #position normalization
    averX = normalInputFile['normalX'].mean()
    averY = normalInputFile['normalY'].mean()

    normalInputFile['normalX'] = normalInputFile['normalX'] - averX
    normalInputFile['normalY'] = normalInputFile['normalY'] - averY
    
    return normalInputFile


#length normalization
def samelen(data,length=400):    
    data_size = len(data)
    interval = (data_size-1)/length
    start = 0
    new_data = data.drop(index=data.index)
    for dist in np.arange(start, data_size-1, interval):
        first = math.floor(dist)
        second = math.ceil(dist)
        if second >= data_size:
            second = data_size -1
        percent = (dist - first) / 1.0   
        #'X','Y','TStamp','Pres','EndPts'
#         print ((data.X[first] + data.X[second])/2)

#         X = (data.at[first,'X'] + data.at[second,'X']) / 2
#         Y = (data.Y[first] + data.Y[second])/2 
        TStamp = (data.TStamp[first] + data.TStamp[second])/2 
        
        X = data.at[first,'X'] + (data.at[second,'X'] - data.at[first,'X']) * percent
        Y = data.at[first,'Y'] + (data.at[second,'Y'] - data.at[first,'Y']) * percent
        normalX = data.at[first,'normalX'] + (data.at[second,'normalX'] - data.at[first,'normalX']) * percent
        normalY = data.at[first,'normalY'] + (data.at[second,'normalY'] - data.at[first,'normalY']) * percent 
        TStamp2 = data.at[first,'TStamp2'] + (data.at[second,'TStamp2'] - data.at[first,'TStamp2']) * percent 
        Pres = data.at[first,'Pres'] + (data.at[second,'Pres'] - data.at[first,'Pres']) * percent 
        Angle = data.at[first,'Angle'] + (data.at[second,'Angle'] - data.at[first,'Angle']) * percent
        Vel = data.at[first,'Vel'] + (data.at[second,'Vel'] - data.at[first,'Vel']) * percent 
        Logcr = data.at[first,'Logcr'] + (data.at[second,'Logcr'] - data.at[first,'Logcr']) * percent 
        Tam = data.at[first,'Tam'] + (data.at[second,'Tam'] - data.at[first,'Tam']) * percent 
        
        d_X = data.at[first,'d_X'] + (data.at[second,'d_X'] - data.at[first,'d_X']) * percent
        d_Y = data.at[first,'d_Y'] + (data.at[second,'d_Y'] - data.at[first,'d_Y']) * percent 
        d_Angle = data.at[first,'d_Angle'] + (data.at[second,'d_Angle'] - data.at[first,'d_Angle']) * percent
        d_Vel = data.at[first,'d_Vel'] + (data.at[second,'d_Vel'] - data.at[first,'d_Vel']) * percent 
        d_Logcr = data.at[first,'d_Logcr'] + (data.at[second,'d_Logcr'] - data.at[first,'d_Logcr']) * percent 
        d_Tam = data.at[first,'d_Tam'] + (data.at[second,'d_Tam'] - data.at[first,'d_Tam']) * percent 
        
        if data.EndPts[first] == 1 or data.EndPts[second] == 1:
            EndPts = 1
        else:
            EndPts = 0
            
        new_data.loc[len(new_data)] = {'X': X, 'Y': Y, 'TStamp': TStamp, 'Pres': Pres, 'EndPts': EndPts,'normalX': normalX,
                         'normalY': normalY, 'TStamp2': TStamp2,'Angle': Angle,'Vel': Vel,'Logcr': Logcr,'Tam': Tam,
                         'd_X': d_X, 'd_Y': d_Y,'d_Angle': d_Angle,'d_Vel': d_Vel,'d_Logcr': d_Logcr,'d_Tam': d_Tam,} 
    return new_data

In [7]:
"""
    add new features to signals
"""

def addSigFeature(df):
    dX = derivation(df['X'].tolist())     # dx
    dY = derivation(df['Y'].tolist())     # dy
#     df['dX'] = dX
#     df['dY'] = dY
    Vel = np.zeros(len(df)).tolist()   # velocity
    Angle = np.zeros(len(df)) # angle
    
    
    T = len(df) -1
    t = 1
    while t<= T:
        Vel[t] = math.sqrt(dX[t] * dX[t] + dY[t] * dY[t])
        if dY[t] != 0 and dX[t] != 0:
            Angle[t] = math.atan(dY[t] / dX[t])
        elif dX[t] == 0:
            Angle[t] = math.atan(dY[t] / 0.01)
        else:
            Angle[t] = 0
        t += 1    
    dAngle = derivation(Angle)
    dVel = derivation(Vel)
    Logcr = np.zeros(len(df)).tolist() 
    Tam = np.zeros(len(df)).tolist() #加速度
    
    t = 1
    while t<= T:
        Logcr[t] = math.log((abs(Vel[t]) + 0.01) / ((abs(dAngle[t]) + 0.01))) ;
        Tam[t] = math.sqrt(dVel[t] * dVel[t] + Vel[t] * Vel[t] * dAngle[t] * dAngle[t]) ;
        t += 1
    df['Angle'] = Angle
    df['Vel'] = Vel
    df['Logcr'] = Logcr
    df['Tam'] = Tam
    
    return df
    
    
# calculate the difference of the discrete sequence    
def derivation(signal):
    T = len(signal)-1
    dsignal = np.zeros(len(signal)).tolist()
    dsignal[0] = (2*signal[2] + signal[1] - 3*signal[0]) / 5
    dsignal[1] = (2*signal[3] + signal[2] - 2*signal[1] - signal[0]) / 6
    t= 2
    while t<=T-2:
        dsignal[t] = (2*signal[t+2] + signal[t+1] - signal[t-1] - 2*signal[t-2]) / 10 
#         print(dsignal[t])        
        t += 1
        # (2(Xn+2)+(Xn+1)-(Xn-1)-2(Xn-2))/10 == [2(Xn+2)-2(Xn+1)+3(Xn+1)-3(Xn)+3(Xn)-3(Xn-1)+2(Xn-1)-2(Xn-2)]/10
        
    dsignal[T-1] = (signal[T] - signal[T-2] + 2*signal[T-1] - 2*signal[T-3]) / 6 
    dsignal[T] = (3*signal[T] - signal[T-1] - 2*signal[T-2]) / 5 
    return dsignal

# calculate the first order derivation for all 6 features   
def addFirstOrderDerivation(df):
    d_X = derivation(df['X'].tolist())     
    d_Y = derivation(df['Y'].tolist())     
    d_Angle = derivation(df['Angle'].tolist())   
    d_Vel = derivation(df['Vel'].tolist())   
    d_Logcr = derivation(df['Logcr'].tolist())   
    d_Tam = derivation(df['Tam'].tolist())   
    
    df['d_X'] = d_X
    df['d_Y'] = d_Y
    df['d_Angle'] = d_Angle
    df['d_Vel'] = d_Vel 
    df['d_Logcr'] = d_Logcr 
    df['d_Tam'] = d_Tam     
    return df

In [8]:
"""
    calculate the euclidean distance using DTW of two sequences 
"""
def calDistance():
    #set ts_a
    normalX = np.array(data1['normalX'])
    normalY = np.array(data1['normalY'])
    list(zip(normalX,normalY))
    ts_a = np.array(list(zip(normalX,normalY)),dtype=float)
#     print(normalX)

    #set ts_b
    normalX = np.array(data2['normalX'])
    normalY = np.array(data2['normalY'])
    list(zip(normalX,normalY))
    ts_b = np.array(list(zip(normalX,normalY)),dtype=float)

    #set ts_c
    normalX = np.array(data3['normalX'])
    normalY = np.array(data3['normalY'])
    list(zip(normalX,normalY))
    ts_c = np.array(list(zip(normalX,normalY)),dtype=float)


    #the plot of DTW distance for genuine signature
    distance, path = fastdtw(ts_a, ts_b, dist=euclidean)
    print("genuine signature distance: ", distance)    

In [9]:
#preprocessing
def preprocess(df, length=400):
    startTime = df['TStamp'][0]
    df['TStamp2'] = (df['TStamp'] - startTime) #ms
#     df['EndPts'][len(df)-1] = 1
#     print(df)
#     df = rdp_precoss(df) # use RDP algorithm to remove duplicated points
#     df = interpolate_points(df)
    df = remove_duplicated_point(df)
    df = addEmptyFeature(df)    
    df = addSigFeature(df)
    df = addFirstOrderDerivation(df)    
#     df['normalX'] = df['X']
#     df['normalY'] = df['Y']    
    df = normalizeFile(df)
    #     df = rotate_graphic(df)    
    
    df = samelen(df, length)
    df = normalization_df(df)
    return df

def get_ts(df,names=['angle','vel','logcr','tam']):
    normalX = np.array(df['normalX'])
    normalY = np.array(df['normalY'])
    X = np.array(df['X'])
    Y = np.array(df['Y'])    
    Pres = np.array(df['Pres'])
    Angle = np.array(df['Angle'])
    Vel = np.array(df['Vel'])
    Logcr = np.array(df['Logcr'])
    Tam = np.array(df['Tam'])
    
    d_X = np.array(df['d_X'])
    d_Y = np.array(df['d_Y'])
    d_Angle = np.array(df['d_Angle'])
    d_Vel = np.array(df['d_Vel'])
    d_Logcr = np.array(df['d_Logcr'])
    d_Tam = np.array(df['d_Tam'])    
    
    features = [normalX,normalY]
    if 'angle' in names:
        features.append(Angle)
    if 'vel' in names:
        features.append(Vel)
    if 'logcr' in names:
        features.append(Logcr)
    if 'tam' in names:
        features.append(Tam)        
#     ts = np.array(list(zip(*features)),dtype=float)   
#     ts = np.array(list(zip(normalX,normalY,Angle,Vel,Logcr,Tam)),dtype=float) 
    ts = np.array(list(zip(normalX,normalY,Angle,Vel,Logcr,Tam,d_X,d_Y,d_Angle,d_Vel,d_Logcr,d_Tam)),dtype=float) 
#     ts = np.array(list(zip(normalX,normalY,Angle,Vel,Logcr,Tam,)),dtype=float) 
  
    return ts

def cal_dfs_max_length(path, files):
    max_length = 0
    for fname in files:
        with open(path + '/' + fname,'rb') as f:
            count = 0
            last_data = '\n'
            while True:
                data = f.read(0x400000)
                if not data:
                    break
                count += data.count(b'\n')
                last_data = data
            if last_data[-1:] != b'\n':
                count += 1 # Remove this if a wc-like count is needed        
        length = count
        if length > max_length:
            max_length = length
    print(max_length-2)        
    return max_length-2  

def normalization_df(df):
    TStamp2 = df['TStamp2'].tolist()
    TStamp = df['TStamp'].tolist()
    normalized_df=(df-df.mean())/df.std()
#     normalized_df=(df-df.min())/(df.max()-df.min())
    normalized_df['TStamp2'] = TStamp2
    normalized_df['TStamp'] = TStamp
    return normalized_df

# def get_all_feature_combi(all_feature_names = ['angle','vel','logcr','tam']):
#     all_feature_names = ['angle','vel','logcr','tam']
# #     all_feature_combi = [['angle'],['vel'],['logcr'],['tam'],
# #                          ['angle','vel'],['angle','logcr'],['angle','tam'],['vel','logcr'],['vel','tam'],['logcr','tam'],
# #                          ['angle','vel','logcr'],['angle','vel','tam'],['vel','logcr','tam'],
# #                          ['angle','vel','logcr','tam']]
#     all_feature_combi = [all_feature_names]    
#     return all_feature_combi

def addEmptyFeature(df):
    df['d_X'] = 0
    df['d_Y'] = 0
    df['d_Angle'] = 0
    df['d_Vel'] = 0 
    df['d_Logcr'] = 0 
    df['d_Tam'] = 0
    return df

In [10]:
# # read dataset

# base_path_forg = 'dataset/SUSIG/FORGERY' #FROM FILE PATH
# base_path_genu = 'dataset/SUSIG/GENUINE/SESSION2' #FROM FILE PATH
# save_base_path_forg = 'dataset/SUSIG/FEATURE2/FORGERY' #SAVE FILE PATH
# save_base_path_genu = 'dataset/SUSIG/FEATURE2/GENUINE/SESSION2' #SAVE FILE PATH


# DEBUG_MODE = True
# TOTAL_NUM = 0.0
# TOTAL_GENU = 0.0
# TOTAL_FORG = 0.0
# TOTAL_FN = 0.0
# TOTAL_FP = 0.0
# TOTAL_ACCU = 0.0
# TRAINING_SET_SIZE = 3
# TOTAL_FEATURE_NUM = 12
# counter = 1

# # while (counter<116):
# while (counter<116):    
#     if (counter<10):
#         base1 = "00"
#     elif (counter <100):
#         base1 = "0"
#     else:
#         base1 = ""
        
#     genuinefiles = []
#     forgeryfiles = []
        
#     #get genuine signature    
#     for root, dirs, files in os.walk(base_path_genu):
#         for f in files:
#             if f.startswith(base1 + str(counter)):
#                 genuinefiles = np.append(genuinefiles,f)
#                 genuinefiles = list(genuinefiles)
#     print(genuinefiles)
#     #get forgery signature    
#     for root, dirs, files in os.walk(base_path_forg):
#         for f in files:
#             if f.startswith(base1 + str(counter)):
#                 forgeryfiles = np.append(forgeryfiles,f)
#                 forgeryfiles = list(forgeryfiles)
#     if len(genuinefiles) == 0:
#         counter += 1
#         continue
        
#     # feature extraction , preprocessing and save
#     dfs_max_length = cal_dfs_max_length(base_path_genu, genuinefiles)
#     for f in genuinefiles:
#         temp = str(base_path_genu+"/"+f)
#         df = pd.read_csv(temp,delimiter=' ', names=['X','Y','TStamp','Pres','EndPts'], header=None, skiprows=2)        
#         df = preprocess(df,dfs_max_length)    
#         df.to_csv(path_or_buf=save_base_path_genu+"/"+f,sep=" ",header=True, index= False)
    
# #     for f in forgeryfiles:
# #         temp = str(base_path_forg+"/"+f)
# #         df = pd.read_csv(temp,delimiter=' ', names=['X','Y','TStamp','Pres','EndPts'], header=None, skiprows=2)        
# #         df = preprocess(df,dfs_max_length)    
# #         df.to_csv(path_or_buf=save_base_path_forg+"/"+f,sep=" ",header=True, index= False)

        
    
#     # X Y TStamp Pres EndPts TStamp2 Angle Vel Logcr Tam normalX normalY d_normalX d_normalY d_Angle d_Vel d_Logcr d_Tam

# #     genu_train_dfs = []
# #     for f in genuinefiles:
# #         temp = str(save_base_path_genu+"/"+f)
# #         df = pd.read_csv(temp,delimiter=' ', names=['X','Y','TStamp','Pres','EndPts','TStamp2','Angle','Vel','Logcr','Tam','normalX','normalY', 
# #                                                     'd_normalX','d_normalY','d_Angle','d_Vel','d_Logcr','d_Tam'], header=None, skiprows=1)        
        
# #         genu_train_dfs.append(df)
        
# #     #calculate the threshold distance from training set
# #     for i in range(0, len(genu_train_dfs)-1):
# #         data1 = genu_train_dfs[i]
# #         for j in range(i+1, len(genu_train_dfs)):
# #             data2 = genu_train_dfs[j]
# #             #set ts_a
# #             ts_a = get_ts(data1)
# #             #set ts_b
# #             ts_b =  get_ts(data2)
# #             distance, path = fastdtw(ts_a, ts_b, radius=TOTAL_FEATURE_NUM, dist=euclidean)
# #             print("genuine signature distance: ", distance)
            
#     counter += 1 
# #     continue    
# #         df = preprocess(df,dfs_max_length)
# #         genu_train_times.append(df['TStamp2'][len(df)-1])
# #         genu_train_dfs.append(df)
    

In [12]:
# read dataset

base_path_forg = 'dataset/SUSIG/FEATURE2/FORGERY' #FROM FILE PATH
base_path_genu = 'dataset/SUSIG/FEATURE2/GENUINE/SESSION1' #FROM FILE PATH
base_path_genu2 = 'dataset/SUSIG/FEATURE2/GENUINE/SESSION2' #FROM FILE PATH

DEBUG_MODE = True
TOTAL_NUM = 0.0
TOTAL_GENU = 0.0
TOTAL_FORG = 0.0
TOTAL_FN = 0.0
TOTAL_FP = 0.0
TOTAL_ACCU = 0.0
TRAINING_SET_SIZE = 3
TOTAL_FEATURE_NUM = 12
FILE_FEATURE_NAMES = ['X','Y','TStamp','Pres','EndPts','TStamp2','d_X','d_Y','d_Angle','d_Vel','d_Logcr','d_Tam',
                      'Angle','Vel','Logcr','Tam','normalX','normalY']
counter = 1

# while (counter<116):
while (counter<116):    
    if (counter<10):
        base1 = "00"
    elif (counter <100):
        base1 = "0"
    else:
        base1 = ""
        
    genuinefiles = []
    genuinefiles2 = []
    forgeryfiles = []
        
    #get genuine signature    
    for root, dirs, files in os.walk(base_path_genu):
        for f in files:
            if f.startswith(base1 + str(counter)):
                genuinefiles = np.append(genuinefiles,f)
                genuinefiles = list(genuinefiles)
                
    
    for root, dirs, files in os.walk(base_path_genu2):
        for f in files:
            if f.startswith(base1 + str(counter)):
                genuinefiles2 = np.append(genuinefiles2,f)
                genuinefiles2 = list(genuinefiles2)                
    print(genuinefiles)
    
    #get forgery signature    
    for root, dirs, files in os.walk(base_path_forg):
        for f in files:
            if f.startswith(base1 + str(counter)):
                forgeryfiles = np.append(forgeryfiles,f)
                forgeryfiles = list(forgeryfiles)
                
    if len(genuinefiles) == 0:
        counter += 1
        continue

    genuinefiles.append(genuinefiles[1])
    genuinefiles.remove(genuinefiles[1])
    
    i = 0
    genu_train_dfs = [] # genuine data
    genu_train_dists = [] # distances between genuine signature
    genu_train_times = [] # times of genuine signature
    genu_train_files = []
    dfs_max_length = 0 # max length(number of points) in trainning dataset

    
    dist_flag = 0 # 0 indicates recalculation
    cal_count = 0
    min_coef_genu_train_dfs = [] # min coef genuine data
    min_coef_genu_train_dists = [] # min coef distances between genuine signature
    min_coef_genu_train_times = [] # min coef times of genuine signature
    min_coef_genu_train_files = []
    min_coef = 1.0 # min coef
    min_coef_chosen_feature_names = []
    chosen_feature_names = []
    
    sample_choose_start_number = 0
    while  len(genu_train_files)==0 or dist_flag == 0:
        genu_train_files = random.sample(genuinefiles, TRAINING_SET_SIZE)
#         genu_train_files = genuinefiles[sample_choose_start_number:(sample_choose_start_number+TRAINING_SET_SIZE)]
        sample_choose_start_number += 1        
        genu_train_dfs = [] # genuine data
        genu_train_dists = [] # distances between genuine signature
        genu_train_times = [] # times of genuine signature
        
        dfs_max_length = cal_dfs_max_length(base_path_genu, genu_train_files)
        for f in genu_train_files:
            temp = str(base_path_genu+"/"+f)
            df = pd.read_csv(temp,delimiter=' ', names=FILE_FEATURE_NAMES, header=None, skiprows=1)        
            genu_train_times.append(df['TStamp2'][len(df)-1])
            genu_train_dfs.append(df)
            
        
        all_feature_names = ['angle','vel','logcr','tam','d_X','d_Y','d_angle','d_vel','d_logcr','d_tam']
        all_feature_combi = [all_feature_names]
#         all_feature_combi = get_all_feature_combi(all_feature_names)
        feature_coef_var_min = 1.0
        chosen_feature_names = []
        for feature in all_feature_combi:
            feature_genu_train_dists = []
            
            #calculate the threshold distance from training set
            for i in range(0, len(genu_train_dfs)-1):
                data1 = genu_train_dfs[i]
                for j in range(i+1, len(genu_train_dfs)):
                    data2 = genu_train_dfs[j]
                    #set ts_a
                    ts_a = get_ts(data1,feature)
                    #set ts_b
                    ts_b =  get_ts(data2,feature)
                    distance, path = fastdtw(ts_a, ts_b, radius=TOTAL_FEATURE_NUM, dist=euclidean)
                    if DEBUG_MODE:
                        print("genuine signature distance: ", distance)
                    feature_genu_train_dists.append(distance)

            # 判断是否训练集的dtw距离都小于规定值，如果不小于重新算
            # Determine whether the dtw distance of the training set is less than the specified value, perform the recalculation if not less than it
            coef_var_temp = np.std(feature_genu_train_dists)/np.mean(feature_genu_train_dists)
#             print('coefficient of variation:',coef_var_temp)
#             print('循环计算的特征值:',feature, "\n")   
            if coef_var_temp < feature_coef_var_min:
                feature_coef_var_min = coef_var_temp
                chosen_feature_names = feature
                genu_train_dists = feature_genu_train_dists
              # If the coefficient of variation is too large, it is definitely not a suitable training set, just skip  
#             if  coef_var_temp > 0.35: 
#                 break
                
        
        coef_var = feature_coef_var_min 
        print('the chosen coefficient of varation:',coef_var)
#         print('选择的特征值:',chosen_feature_names)        
        dist_flag = 1
        cal_count += 1        
        # 计算并保存一个最小的coef值，如果超过最大的cal_count值，则直接把这个值所表示的训练集当做真正的训练集
        # Calculate and save a minimum coef value. If it exceeds the maximum cal_count value, 
        # the training set of this coef value is directly regarded as the real training set
        if coef_var < min_coef:
            min_coef_genu_train_dfs = genu_train_dfs
            min_coef_genu_train_dists = genu_train_dists
            min_coef_genu_train_times = genu_train_times
            min_coef_genu_train_files = genu_train_files
            min_coef = coef_var
            min_coef_chosen_feature_names = chosen_feature_names
            
        
        # 如果离散系数大于0.15 或者 (最大值-最小值)/平均值 > 0.2 重新选训练集
        # If the coefficient of variation is greater than 0.13 or (maximum-minimum)/average value> 0.2, reselect the training set
        if coef_var > 0.15 :
            dist_flag = 0
            print ("The coefficient of varation is too high")
        elif (max(genu_train_dists) - min(genu_train_dists))/((max(genu_train_dists) + min(genu_train_dists))/2)>0.25:
            dist_flag = 0
            print("The extreme value coefficient is too high",max(genu_train_dists),min(genu_train_dists),
                  (max(genu_train_dists) - min(genu_train_dists))/((max(genu_train_dists) + min(genu_train_dists))/2)) 
        
        # 超过了max_cal_count的话，就直接选最小的coef,不再继续算了
        # If it has tried more than the max_cal_count, just choose the smallest coef instead of calculating further
        if dist_flag == 0 and cal_count >= 5:             
            genu_train_dfs = min_coef_genu_train_dfs
            genu_train_dists  = min_coef_genu_train_dists
            genu_train_times  = min_coef_genu_train_times
            genu_train_files  = min_coef_genu_train_files
            chosen_feature_names = min_coef_chosen_feature_names
            dist_flag = 1
#                 break
#     counter = counter + 1
#     continue
    
    
    
                       
    min_train_dist = min(genu_train_dists)
    mean_train_dist = np.mean(genu_train_dists)
#     threshold = min_train_dist * 1.2
    print(np.std(genu_train_dists))
    
    buffer = 0
    if np.std(genu_train_dists) * 5 < 0.5 * mean_train_dist:
        buffer = 0.5 * mean_train_dist
        print("mean is chosen as buffer")
    else:    
        buffer = np.std(genu_train_dists) * 5
        print("standard deviation is chosen as buffer")
        
        
    threshold = mean_train_dist + buffer  
    threshold_time = max(genu_train_times)*2 # maximum time of genuine siganature in training set
    
    
    test_dfs = []
    total_num = len(genuinefiles) + len(genuinefiles2) + len(forgeryfiles) 
    TP = 0.0
    FP = 0.0 
    FN = 0.0
    TN = 0.0
    
    for f in genuinefiles:
        if f in genu_train_files: # skip samples in trainning set
            total_num -= 1
            continue
        print(f)    
        temp = str(base_path_genu+"/"+f)
        df = pd.read_csv(temp,delimiter=' ', names=FILE_FEATURE_NAMES, header=None, skiprows=1) 
        if (df['TStamp'].max() - df['TStamp'].min()) > threshold_time:
            FN += 1
            continue
        test_dfs.append(df) 
 
    for f in genuinefiles2:
        print(f)    
        temp = str(base_path_genu2+"/"+f)
        df = pd.read_csv(temp,delimiter=' ', names=FILE_FEATURE_NAMES, header=None, skiprows=1) 
        if (df['TStamp'].max() - df['TStamp'].min()) > threshold_time:
            FN += 1
            continue
        test_dfs.append(df) 
    test_dfs_geni_num = len(test_dfs)
    
    for f in forgeryfiles:
        temp = str(base_path_forg+"/"+f)
        print(f) 
        df = pd.read_csv(temp,delimiter=' ', names=FILE_FEATURE_NAMES, header=None, skiprows=1)
        if (df['TStamp'].max() - df['TStamp'].min()) > threshold_time:
            TN += 1
            continue
#         df = preprocess(df,dfs_max_length)
        test_dfs.append(df)
    
    
    #calculate the min distance from each test to training set, if d <= threshold, we say it is genuine, otherwise forgery 
    for i in range(0, len(test_dfs)):
        data1 = test_dfs[i]
        test_dists = []
        for j in range(0, len(genu_train_dfs)):
            data2 = genu_train_dfs[j]
            #set ts_a
            ts_a =  get_ts(data1, chosen_feature_names)
            #set ts_b
            ts_b =  get_ts(data2, chosen_feature_names)
            distance, path = fastdtw(ts_a, ts_b, radius=TOTAL_FEATURE_NUM, dist=euclidean)
            test_dists.append(distance)
            
        min_test_dist = min(test_dists)
        max_test_dist = max(test_dists)
        mean_test_dist = np.mean(test_dists)
        test_dist_chosen = mean_test_dist # choose the mean of test distances
        if min_test_dist == 0:
            total_num -= 1
            print(total_num)
            continue
            
        if DEBUG_MODE:    
            print("test signature min distance: ", min_test_dist," mean distance: ", mean_test_dist, " threshold: ", threshold)
        if test_dist_chosen > threshold  and i<test_dfs_geni_num:
            FN += 1.0
        elif  test_dist_chosen > threshold and i>=test_dfs_geni_num:
            TN += 1.0
        elif  test_dist_chosen <= threshold and i<test_dfs_geni_num:
            TP += 1.0
        elif  test_dist_chosen <= threshold and i>=test_dfs_geni_num:
            FP += 1.0           
            
            
    genu_test_num = len(genuinefiles) + len(genuinefiles2) - TRAINING_SET_SIZE
    forg_test_num = len(forgeryfiles)
    accuracy = (TP + TN)/total_num * 100
    FRR = FN/genu_test_num *100 
    FAR = FP/forg_test_num *100 
    
    TOTAL_NUM += total_num
    TOTAL_GENU += genu_test_num
    TOTAL_FORG += forg_test_num
    TOTAL_FN += FN
    TOTAL_FP += FP
    TOTAL_FRR = TOTAL_FN/TOTAL_GENU * 100
    TOTAL_FAR = TOTAL_FP/TOTAL_FORG *100
    
    TOTAL_ACCU = (TOTAL_NUM - TOTAL_FN - TOTAL_FP) / TOTAL_NUM * 100
    print("test num: " + str(total_num) + ", accuracy: " + str(accuracy) + "%")
    print("FRR: " + str(FRR) + "%, FAR: " + str(FAR)+ "%")
    print("TOTAL_FRR: " + str(TOTAL_FRR) + "%, TOTAL_FAR: " + str(TOTAL_FAR)+ "%, TOTAL_TEST_SIGN:" + 
          str(TOTAL_NUM/(genu_test_num + forg_test_num)) + "TOTAL_ACCU: " + str(TOTAL_ACCU) + "%")
    print("\n") 
    
    counter = counter + 1

['001_1_1.sig', '001_1_10.sig', '001_1_2.sig', '001_1_3.sig', '001_1_4.sig', '001_1_5.sig', '001_1_6.sig', '001_1_7.sig', '001_1_8.sig', '001_1_9.sig']
175
genuine signature distance:  266.85395561046835
genuine signature distance:  237.36432532638554
genuine signature distance:  311.15760043986194
the chosen coefficient of varation: 0.11158406855707292
The extreme value coefficient is too high 311.15760043986194 237.36432532638554 0.26906226222549723
178
genuine signature distance:  315.8758929765428
genuine signature distance:  322.72611345748015
genuine signature distance:  326.1062067929274
the chosen coefficient of varation: 0.013234604812066444
4.255843987004473
mean is chosen as buffer
001_1_2.sig
001_1_4.sig
001_1_6.sig
001_1_7.sig
001_1_8.sig
001_1_9.sig
001_1_10.sig
001_2_1.sig
001_2_10.sig
001_2_2.sig
001_2_3.sig
001_2_4.sig
001_2_5.sig
001_2_6.sig
001_2_7.sig
001_2_8.sig
001_2_9.sig
001_f_1.sig
001_f_10.sig
001_f_2.sig
001_f_3.sig
001_f_4.sig
001_f_5.sig
001_f_6.sig
001_f_7

test signature min distance:  156.63400849393005  mean distance:  162.93690260765763  threshold:  276.6371153544464
test signature min distance:  174.09740484131495  mean distance:  181.31779451137334  threshold:  276.6371153544464
test signature min distance:  171.62444008031795  mean distance:  185.2867860254813  threshold:  276.6371153544464
test signature min distance:  177.7090930971165  mean distance:  186.0346371338591  threshold:  276.6371153544464
test signature min distance:  149.1011211972688  mean distance:  163.24162628431455  threshold:  276.6371153544464
test signature min distance:  177.7311519142316  mean distance:  182.14560516070983  threshold:  276.6371153544464
test signature min distance:  193.66573525193886  mean distance:  200.53068244428536  threshold:  276.6371153544464
test signature min distance:  189.15132765968718  mean distance:  200.83676478566508  threshold:  276.6371153544464
test signature min distance:  192.88508322748459  mean distance:  193.6399622

test signature min distance:  251.99464804790844  mean distance:  278.8164865097615  threshold:  405.69027586070763
test signature min distance:  224.69978306799422  mean distance:  269.2399631912985  threshold:  405.69027586070763
test signature min distance:  194.06158564664617  mean distance:  240.60700941784512  threshold:  405.69027586070763
test signature min distance:  278.35323880734506  mean distance:  286.2559980940541  threshold:  405.69027586070763
test signature min distance:  298.9913485814245  mean distance:  335.0784336354646  threshold:  405.69027586070763
test signature min distance:  366.06862778345993  mean distance:  397.40098828278633  threshold:  405.69027586070763
test signature min distance:  424.8687698684345  mean distance:  454.9388661453345  threshold:  405.69027586070763
test signature min distance:  379.9748698341627  mean distance:  425.6701347741461  threshold:  405.69027586070763
test signature min distance:  465.78806795229474  mean distance:  494.624

test signature min distance:  475.4501473943407  mean distance:  529.9535515391059  threshold:  764.6525276276285
test signature min distance:  525.2283202702297  mean distance:  564.0727116155757  threshold:  764.6525276276285
test signature min distance:  540.5766211650669  mean distance:  602.4536652590414  threshold:  764.6525276276285
test signature min distance:  530.8878938233051  mean distance:  552.9664585508352  threshold:  764.6525276276285
test signature min distance:  556.1523975579086  mean distance:  583.567465885333  threshold:  764.6525276276285
test signature min distance:  465.4734108217764  mean distance:  513.0191188548921  threshold:  764.6525276276285
test signature min distance:  490.07139936244016  mean distance:  517.4147414232733  threshold:  764.6525276276285
test signature min distance:  569.7330503957362  mean distance:  615.3057551588623  threshold:  764.6525276276285
test signature min distance:  604.307705224014  mean distance:  643.345004391334  thresh

test signature min distance:  579.1713572857102  mean distance:  684.8345044610945  threshold:  796.6056747893883
test signature min distance:  591.2357647988263  mean distance:  656.8771022368388  threshold:  796.6056747893883
test signature min distance:  694.3648526322361  mean distance:  737.362678047134  threshold:  796.6056747893883
test signature min distance:  747.4261044350814  mean distance:  800.5480328219036  threshold:  796.6056747893883
test signature min distance:  663.1930615505721  mean distance:  735.8011509751092  threshold:  796.6056747893883
test signature min distance:  623.0731998318165  mean distance:  720.03333975169  threshold:  796.6056747893883
test signature min distance:  569.97642495419  mean distance:  651.1136571101673  threshold:  796.6056747893883
test signature min distance:  868.2564156151836  mean distance:  916.5502884111069  threshold:  796.6056747893883
test signature min distance:  996.1781448701157  mean distance:  1007.7524691160038  threshol

...

test signature min distance:  341.06111152674214  mean distance:  373.4382887959518  threshold:  359.5212937536993
test signature min distance:  299.32467819231954  mean distance:  313.56902726751156  threshold:  359.5212937536993
test signature min distance:  248.642312757959  mean distance:  312.92900966497746  threshold:  359.5212937536993
test signature min distance:  312.6229162568598  mean distance:  330.2681876399315  threshold:  359.5212937536993
test signature min distance:  259.7510438357574  mean distance:  315.12076753412924  threshold:  359.5212937536993
test signature min distance:  289.4844519130602  mean distance:  319.57632114373064  threshold:  359.5212937536993
test signature min distance:  351.5953774479756  mean distance:  355.26810716100323  threshold:  359.5212937536993
test signature min distance:  341.4056456562985  mean distance:  368.12581370921816  threshold:  359.5212937536993
test signature min distance:  415.1487783126137  mean distance:  427.323964599349

test signature min distance:  1307.9280774726178  mean distance:  1426.18796326898  threshold:  1546.1151957019672
test signature min distance:  1287.41915414943  mean distance:  1447.184117419297  threshold:  1546.1151957019672


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-bdf5aee3ca39>", line 244, in <module>
    distance, path = fastdtw(ts_a, ts_b, radius=TOTAL_FEATURE_NUM, dist=euclidean)
  File "D:\ProgramData\Anaconda3\lib\site-packages\fastdtw\fastdtw.py", line 53, in fastdtw
    return __fastdtw(x, y, radius, dist)
  File "D:\ProgramData\Anaconda3\lib\site-packages\fastdtw\fastdtw.py", line 73, in __fastdtw
    __fastdtw(x_shrinked, y_shrinked, radius=radius, dist=dist)
  File "D:\ProgramData\Anaconda3\lib\site-packages\fastdtw\fastdtw.py", line 75, in __fastdtw
    return __dtw(x, y, window, dist=dist)
  File "D:\ProgramData\Anaconda3\lib\site-packages\fastdtw\fastdtw.py", line 141, in __dtw
    dt = dist(x[i-1], y[j-1])
  File "D:\ProgramData\Anaconda3\lib\site-packages\scipy\spatial\distance.py", line 614, in euclidean
    r

KeyboardInterrupt: 